In [0]:
#@title Params
 

document_parser_lib_version='1.1.12'#@param {type:"string"}
# _git_branch = 'protocols-4'
_git_branch = 'charters'#@param {type:"string"}
# _git_branch = 'semantic-tags'

do_parse_contracts=False#@param {type:"boolean"}
do_parse_protocols=False#@param {type:"boolean"}
do_parse_charters=True#@param {type:"boolean"}

do_parse_zip=False#@param {type:"boolean"}


### Mount gDrive

In [2]:
if do_parse_contracts or do_parse_protocols or do_parse_charters:
  from google.colab import drive
  drive.mount('/content/gdrive' )

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Import code from gitHub

In [3]:
import json
import subprocess
import sys
import time

import numpy as np
from IPython.core.display import display, HTML
from google.colab import files

!pip install overrides

Ничто = None



def exec(x):
  r = subprocess.check_output(x, shell=True)
  r = r.decode('unicode-escape').encode('latin1').decode('utf8')
  print(r)


print(f"fetching code from GitHub.....{_git_branch}")
try:
  exec('rm -r nlp_tools')
except:
  pass
exec(f'git clone --single-branch --branch {_git_branch} https://github.com/nemoware/analyser.git nlp_tools')

print('🦊 GIT revision:')
exec('cd nlp_tools\ngit rev-list --reverse HEAD | awk "{ print NR }" | tail -n 1\ngit branch\ngit log -3 --pretty=%B')

sys.path.insert(0, 'nlp_tools')

print('❤️importing Code from GitHub ... DONE')


#----
import matplotlib as mpl
from documents import TextMap
from colab_support.renderer import HtmlRenderer
 
 

class DemoRenderer(HtmlRenderer):
  def render_color_text(self, tokens, weights, colormap='coolwarm', print_debug=False, _range=None):
    html = self.to_color_text(tokens, weights, colormap, print_debug, _range)
    display(HTML(html))

  def to_color_text(self, tokens, weights, colormap='coolwarm', print_debug=False, _range=None):
    return super()._to_color_text(tokens, weights, mpl, colormap=colormap, _range=_range)

   
renderer_ = DemoRenderer()

 
  

fetching code from GitHub.....charters


🦊 GIT revision:
877
* charters
Fix NPE

Merge branch 'charters-clean' into charters

clean-up cleaning 🧼


❤️importing Code from GitHub ... DONE
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
loading word cases stats model /content/nlp_tools/vocab/word_cases_stats.pickle


### imports

In [4]:
import pickle
import unittest
import warnings

!pip install pyjarowinkler

from contract_parser import ContractAnlysingContext, ContractDocument
from contract_patterns import ContractPatternFactory
from documents import TextMap
from legal_docs import LegalDocument, DocumentJson
from ml_tools import SemanticTag, filter_values_by_key_prefix

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True


 

### Utilits

In [0]:
#@title ✅Util funcs, rendering


def print_semantic_tag(tag: SemanticTag, map: TextMap):
  print('-->', f'{tag.kind} \t\t [{tag.value}]\t [{map.text_range(tag.span)}]')

def print_headers(contract):
  for p in contract.paragraphs:
    print ('\t --> 📂',contract.substr(p.header))

def print_json_summary(cd:DocumentJson):
  wordsmap = TextMap(cd.normal_text, cd.tokenization_maps['$words'])
 
  for tag in cd.tags:
    if tag.kind !='headline':
     print_semantic_tag(tag, wordsmap)
 


def json2html(cd:DocumentJson, render_headlines=False):
  wordsmap = TextMap(cd.normal_text, cd.tokenization_maps['words'])
  markup_vector = np.zeros(len(wordsmap))
 
  if render_headlines:
    for hd in cd.headers:    
      span = hd['span']      
      markup_vector[span[0]:span[1]] += 0.2

  for tag in cd.attributes:
    if 'span' in  cd.attributes[tag]:
      span = cd.attributes[tag]['span']
      confidence = cd.attributes[tag]['confidence']
      markup_vector[span[0]:span[1]] += confidence
    else:
      warnings.warn(f'{tag} has no span')

  return renderer_.to_color_text(wordsmap.tokens, markup_vector)
  
 
def load_and_show_json(filename, render_headlines=False):
  with open(filename, 'r') as json_file:
    data_str = json_file.read()
    # print(str(data_str))
    json_obj:DocumentJson = DocumentJson.from_json( data_str)
    html = json2html(json_obj, render_headlines=render_headlines)
    display(HTML(html))




In [6]:
from tf_support.embedder_elmo import ElmoEmbedder
elmo_embedder = ElmoEmbedder()
elmo_embedder_default = ElmoEmbedder(layer_name="default")
# import tensorflow as tf
# import tensorflow_hub as hub
 
from contract_parser import ContractAnlysingContext
contractAnlysingContext = ContractAnlysingContext(elmo_embedder, renderer_)



INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


### Init document-parser lib

In [0]:
lib_version = document_parser_lib_version
import os
if not os.path.isfile(f'document-parser-{lib_version}-distribution.zip'):
  !wget https://github.com/nemoware/document-parser/releases/download/$lib_version/document-parser-$lib_version-distribution.zip
if not os.path.isdir(f'document-parser-{lib_version}'):
  !unzip document-parser-$lib_version-distribution.zip

 
os.environ ['documentparser']=f'/content/document-parser-{lib_version}'
from integration.word_document_parser import WordDocParser, join_paragraphs


# Export all to JSON

#### Parse all

In [0]:
attempt = 0

In [0]:
import pandas as pd
import hyperparams as hp

codebasepath = os.path.realpath(os.path.join(os.getcwd(), os.path.dirname(hp.__file__)))
expected = pd.read_csv ( os.path.join(codebasepath, 'vocab', 'contracts.validation.csv' ) )

contracts_filename_prefix = '/content/gdrive/My Drive/GazpromOil/Contracts/'

In [10]:
import datetime
import os

attempt += 1
dirname = '{0:%y.%m.%d}'.format(datetime.datetime.now())
_out = f'/content/gdrive/My Drive/GazpromOil/Всяческая документация (shared)/JSONs/{_git_branch}-{dirname}-{attempt}/'


if not os.path.exists(_out):
  os.makedirs(_out)

print(_out)

/content/gdrive/My Drive/GazpromOil/Всяческая документация (shared)/JSONs/charters-19.11.27-1/


In [0]:
import math
import numbers
import os

import pandas as pd

import hyperparams as hp
from legal_docs import DocumentJson


def validate_contracts():
  codebasepath = os.path.realpath(os.path.join(os.getcwd(), os.path.dirname(hp.__file__)))
  expected = pd.read_csv(os.path.join(codebasepath, 'vocab', 'contracts.validation.csv'))
  actual = pd.read_csv(os.path.join(_out, 'contracts.validation.csv'))

  msgs = []
  errors = 0

  def _check(key, e, a, col) -> int:
    _x1 = e[col]
    _x2 = a[col]
    if (isinstance(_x1, numbers.Number) and math.isnan(_x1)) and (isinstance(_x2, numbers.Number) and math.isnan(_x2)):
      return 0
    if _x1 != _x2:
      msg = f"{key}:\t{col} \n\t expected: '{e[col]}' \n\t   ACTUAL: '{a[col]}'"
      msgs.append(msg)
      return 1
    return 0

  for x in actual['_id']:
    r1 = expected[expected['_id'] == x].iloc[0]
    r2 = actual[actual['_id'] == x].iloc[0]

    errors += _check(x, r1, r2, 'org_1_name')
    errors += _check(x, r1, r2, 'org_2_name')
    errors += _check(x, r1, r2, 'value')
    errors += _check(x, r1, r2, 'subject')

  print('=' * 100)
  print('^' * 100)
  print(f"ERRRORS: \t{errors} of {len(actual)} docs")

  with open(os.path.join(_out, 'errors.txt'), 'w') as the_file:
    for m in msgs:
      the_file.write(m + '\n')

    the_file.write('^' * 100)
    the_file.write(f"\nERRRORS: \t{errors} of {len(actual)} docs")
    print(f'errors saved to {the_file}')
  return msgs


def __attr_val(d, a):
  if a in d:
    return d[a]['value']


def to_csv(filenames_to_jsons):
  df = pd.DataFrame(columns=('_id', 'value', 'org_1_name', 'org_2_name', 'subject'))

  i = 0
  for fn in filenames_to_jsons.keys():
    for json_fn in filenames_to_jsons[fn]:
      # print(json_fn)
      with open(json_fn, 'r') as json_file:
        data_str = json_file.read()
        json_obj: DocumentJson = DocumentJson.from_json(data_str)
        # _id =os.path.relpath( json_obj._id, contracts_filename_prefix )
        _id = json_obj._id
        df.loc[i] = [_id,
                     __attr_val(json_obj.attributes, 'sign-value-currency_value'),
                     __attr_val(json_obj.attributes, 'org-1-name'),
                     __attr_val(json_obj.attributes, 'org-2-name'),
                     __attr_val(json_obj.attributes, 'subject')]
        i += 1

  df.to_csv(os.path.join(_out, 'contracts.validation.csv'), encoding='utf-8', index=False)
  return df

# validate_contracts()


In [0]:
import os

from integration.word_document_parser import join_paragraphs, WordDocParser
from legal_docs import DocumentJson


filenames_to_jsons = {}

def for_each_document(filenames, accept, procedure, _out, save_json=True):
  _common_path = os.path.commonpath (filenames)
  
  wp = WordDocParser()
  cnt = 0
  for fn in filenames:
    filenames_to_jsons[fn] = []
    cnt += 1
    _relative = os.path.relpath(fn, _common_path )
    short_fn = os.path.split (_relative)[-1]
    subdir = os.path.dirname(_relative)
    
    # try:
    # ------------------------
    print('\n\n' )
    print('=' * 100)
    print(f'reading:"{fn}"')      
    results = wp.read_doc(fn)
    print("documents in file:", len(results['documents']))
    k = 0
    for doc in results['documents']:  # XXX
      k += 1
      contract_id = _relative
      if accept(doc):
        
        legal_doc = join_paragraphs(doc, contract_id)
        # ------------------------
        legal_doc = procedure(legal_doc)
        assert legal_doc is not None

        if save_json:
          _dirr  = os.path.join(_out, subdir)
          print('_dirr',_dirr)
          _path = os.path.join(_dirr, f'{short_fn}--{k}.json')
          print('_path',_path)
          if not os.path.exists(_dirr):
            os.makedirs(_dirr)

          with open(_path, 'w') as file:
            #
            jjj = DocumentJson(legal_doc)
            file.write(jjj.dumps())
            filenames_to_jsons[fn].append(_path)
            print(f'--> 📂saved file to {_path}')
            print('^' * 100)

    # except Exception as e:
    #   print(e)
   

### 🙏 Contracts: to JSON

In [0]:
def process_contract(contract:LegalDocument) -> LegalDocument:  
  contractAnlysingContext.analyze_contract_doc(contract)  
  print_headers(contract)

  to_csv(filenames_to_jsons)  # TODO: do not read all here
  validate_contracts()
 
  return contract

# 🧠 Contracts
Reading only short list of docs listed in the validation CSV

In [0]:
contract_filenames = [os.path.join(contracts_filename_prefix, fn) for fn in expected['_id']]

In [0]:
# raise 'stop if not sure'

In [0]:
if do_parse_contracts:
  for_each_document(contract_filenames, accept=lambda doc: 'CONTRACT' == doc['documentType'], procedure=process_contract,
                    _out=_out )


In [17]:
df = to_csv(filenames_to_jsons)
from IPython.display import display, HTML

display(HTML(df.to_html(index=False)))

_id,value,org_1_name,org_2_name,subject


## 👌 Validation

In [0]:
if do_parse_contracts:
  validate_contracts()

### Verify random json files

In [19]:
# if do_parse_contracts:
  # test_json_file = os.path.join(_out,filenames[3]).split('/')[-1]+'.json'
  # test_json_file = '/content/gdrive/My Drive/GazpromOil/Всяческая документация (shared)/JSONs/contracts-19.10.17-4/1-! Договор _1.docx.json'
  # test_json_file = os.path.join(_out, '1-ДД_6.docx.json')
  # test_json_file = os.path.join(_out, '1-Договор 15.doc.json')

load_and_show_json('/content/gdrive/My Drive/GazpromOil/Всяческая документация (shared)/JSONs/charters-19.11.19-1/7. Протокол заочного СД ГПН-РП БГ.docx--1.json')
 

In [0]:
# 

# 🧠 Protocols

In [21]:
from google.colab import drive
drive.mount('/content/gdrive' )

from protocol_parser import   ProtocolDocument, ProtocolParser, find_protocol_org
from hyperparams import HyperParameters

def print_tags(doc):
  for t in doc.get_tags():
    print(t.get_key(), t.value)

def process_protocol(doc: LegalDocument) -> LegalDocument:
  protocol_analyser.analyse(doc)
  print('print_tags---------')
  print_tags(doc)
  # print_headers(doc)

  print('find_protocol_org---------')
  print(find_protocol_org(doc))
  return doc

protocols_filename_prefix = '/content/gdrive/My Drive/GazpromOil/Protocols'
protocols_filenames =  WordDocParser().list_filenames(protocols_filename_prefix)

if do_parse_protocols:
  protocol_analyser = ProtocolParser(elmo_embedder, elmo_embedder_default)
  
  # protocols_filenames = ['/content/gdrive/My Drive/GazpromOil/Protocols/7. Протокол заочного СД ГПН-РП БГ.docx']
  for_each_document(protocols_filenames, accept=lambda doc: 'PROTOCOL' == doc['documentType'], procedure=process_protocol,
                    _out=_out, save_json=True)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/document-parser-1.1.12/classes:/content/document-parser-1.1.12/lib/*


In [22]:
protocols_filenames[5:8]

['/content/gdrive/My Drive/GazpromOil/Protocols/6. Протокол заочного СД ГПН-РП.docx',
 '/content/gdrive/My Drive/GazpromOil/Protocols/7. Протокол заочного СД ГПН-РП БГ.docx',
 '/content/gdrive/My Drive/GazpromOil/Protocols/Протокол ГПН-Транспорт 24.04.18.docx']

In [0]:
# raise 'stop here'

# Process contracts zip

In [0]:
#@title Params
 

zip_file_name='contract-protocol-pairs'#@param {type:"string"}
 
out_dir=f'{zip_file_name}-out'
if do_parse_zip:
  
  zipfn=f'/content/{zip_file_name}.zip'
  # zipfn = '/content/contract-protocol-pairs.zip'
  !unzip $zipfn -d $zip_file_name
  !mkdir $out_dir

In [25]:
import glob
 
# zippp = 'contract-protocol-pairs'
filenames = [file for file in glob.glob( f"/content/{zip_file_name}/**/*.doc*", recursive=True)] 
filenames[22:24]

[]

In [0]:
from protocol_parser import   ProtocolDocument, ProtocolParser
from hyperparams import HyperParameters

if do_parse_zip:

  protocol_analyser = ProtocolParser(elmo_embedder, elmo_embedder_default)

  ### TAKES TIME!!!
  for_each_document(filenames, accept=lambda doc: 'PROTOCOL' == doc['documentType'], procedure=process_protocol,
                    _out=out_dir)

In [0]:
if do_parse_zip:
  load_and_show_json('contract-protocol-pairs-out/Крупная сделка(7)/1_Протокол_Тюмень.doc--1.json')

In [0]:
def process_zip_contract(contract:LegalDocument) -> LegalDocument:  
  contractAnlysingContext.analyze_contract_doc(contract)  
  print_headers(contract)  
  return contract

### TAKES TIME!!!
if do_parse_zip:
  for_each_document( filenames, accept=lambda doc: 'CONTRACT' == doc['documentType'], procedure=process_zip_contract,
                    _out=out_dir )

In [0]:
if do_parse_zip:
  load_and_show_json('contract-protocol-pairs-out/Регистратор_обезличен (1)/Договор_ГПН-Терминал.docx--1.json')

zip it

In [0]:
import zipfile
import os
import sys

def zipfolder(foldername, target_dir):            
  zipobj = zipfile.ZipFile(foldername + '.zip', 'w', zipfile.ZIP_DEFLATED)
  rootlen = len(target_dir) + 1
  for base, dirs, files in os.walk(target_dir):
    for file in files:
      fn = os.path.join(base, file)
      zipobj.write(fn, fn[rootlen:])


if do_parse_zip:
  zipfolder(f'{zip_file_name}-out-branch-{_git_branch}', out_dir)

# 🧠 Charters

In [0]:
from charter_parser import CharterParser
charter_parser = CharterParser(elmo_embedder, elmo_embedder_default)

def process_charter(charter: LegalDocument) -> LegalDocument:
  # protocol_analyser.analyse(doc)
  print_headers(charter)
  charter_parser.ebmedd(charter)
  charter_parser.analyse(charter)

  
  return charter

charters_filename_prefix = '/content/gdrive/My Drive/GazpromOil/Charters'
charters_filenames =  WordDocParser().list_filenames(charters_filename_prefix)

if do_parse_charters:
  for_each_document(charters_filenames, accept=lambda doc: 'CHARTER' == doc['documentType'], procedure=process_charter,
                  _out=_out, save_json=True)

In [0]:
  load_and_show_json('/content/gdrive/My Drive/GazpromOil/Всяческая документация (shared)/JSONs/charters-clean-19.11.27-1/яМессояха Устава.docx--1.json')

In [32]:
raise('not sure, not run')

TypeError: ignored

# Picke

In [35]:
# Договор _2_.docx.pickle

def _save_p(FN, doc):
  with open(f'{FN}.pickle', 'wb') as handle:
    pickle.dump(doc, handle, protocol=pickle.HIGHEST_PROTOCOL)




def save_contract(d: LegalDocument) -> LegalDocument:  
  # print(d.__dict__)
  # print_headers(d)
  # charter_parser.ebmedd(charter)
  # charter_parser.analyse(charter)
  # contractAnlysingContext.analyze_contract_doc(d)  
  fn = d._id
  print(fn)
  _save_p(fn, d)
  
  return d

# charters_filename_prefix = '/content/gdrive/My Drive/GazpromOil/Charters'
# charters_filenames =  WordDocParser().list_filenames(charters_filename_prefix)
p_filenames = [
               '/content/gdrive/My Drive/GazpromOil/Contracts/Договор _2_.docx',
               '/content/gdrive/My Drive/GazpromOil/Contracts/Договор 8.docx',
               '/content/gdrive/My Drive/GazpromOil/Contracts/Договор 2.docx',
               '/content/gdrive/My Drive/GazpromOil/Contracts/2. Договор по благ-ти Радуга.docx'
]
if do_parse_charters:
  for_each_document(p_filenames, accept=lambda doc: 'CONTRACT' == doc['documentType'], procedure=save_contract,
                  _out=_out, save_json=False)

/content/document-parser-1.1.12/classes:/content/document-parser-1.1.12/lib/*



reading:"/content/gdrive/My Drive/GazpromOil/Contracts/Договор _2_.docx"
documents in file: 5
Договор _2_.docx



reading:"/content/gdrive/My Drive/GazpromOil/Contracts/Договор 8.docx"
documents in file: 1
Договор 8.docx



reading:"/content/gdrive/My Drive/GazpromOil/Contracts/Договор 2.docx"
documents in file: 1
Договор 2.docx



reading:"/content/gdrive/My Drive/GazpromOil/Contracts/2. Договор по благ-ти Радуга.docx"
documents in file: 1
2. Договор по благ-ти Радуга.docx


#Parse 2K sectioned contracts (Achtung, dis takes time)
- upload zipped results of document-parser first 

In [0]:
from google.colab import files
uploaded = files.upload()
!unzip /content/jsons_phase0.zip
import glob
filenames = [file for file in glob.glob( "/content/jsons_phase0/**/*.json", recursive=True)] 

!mkdir out

In [0]:

import csv, json
from bson import json_util


from contract_parser import ContractAnlysingContext
from integration.word_document_parser import join_paragraphs


def export_csv(rows, headline=['1', '2', '3', '4', '5', '6', '7', '8', '9']):
  with open(f'contracts-stats.csv', mode='w') as csv_file:
    _writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    _writer.writerow(headline)
    for l in rows:
      _writer.writerow(l)


def _parse_contract(res, doc_id, row, ctx: ContractAnlysingContext):
  contract = join_paragraphs(res, doc_id)
  ctx.analyze_contract_doc(contract)

  row[4:8] = [contract.tag_value('org.1.name'),
              contract.tag_value('org.1.alias'),
              contract.tag_value('org.2.name'),
              contract.tag_value('org.2.alias')]
  return contract

 
rows=[]
cnt=0

for fn in filenames[200:]:
  with open(fn, 'r') as file:
    json_string = file.read()
    res = json.loads(json_string, object_hook=json_util.object_hook)

  if 'CONTRACT' == res["documentType"]:
    row = [cnt, '', None, None, None, None, None, None, fn, None]
    row[2:4] = [res["documentType"], res["documentDate"]]
    contract = _parse_contract(res, fn,  row, contractAnlysingContext)
         
    short_fn = fn.split('/')[-1]
    with open(f'/content/out/{short_fn}.json', 'w') as file:
      jjj = DocumentJson(contract)
      file.write(jjj.dumps())


    rows.append(row)
    if cnt % 5 == 0: #save every 5 rows
      export_csv(rows)

  cnt+=1

In [0]:
cnt

In [0]:
import zipfile
import os
import sys

def zipfolder(foldername, target_dir):            
  zipobj = zipfile.ZipFile(foldername + '.zip', 'w', zipfile.ZIP_DEFLATED)
  rootlen = len(target_dir) + 1
  for base, dirs, files in os.walk(target_dir):
    for file in files:
      fn = os.path.join(base, file)
      zipobj.write(fn, fn[rootlen:])



zipfolder('out','/content/out')